In [55]:
import requests
import numpy as np
import pandas as pd
import datetime as dt

pd.set_option('float_format', '{:,.2f}'.format)

# Set globals
API_KEY = 'fbf2a3cac76ec733ee2b8c01ab036950'
URL_BASE = 'https://api.stlouisfed.org/fred/series/observations'
START = pd.Timestamp('1983-01-01').date()
END = pd.Timestamp('2022-12-31').date()
BUSDAYS_IN_RANGE = np.busday_count(START, END)
META_INDEX = ['observation_start', 'observation_end', 'busdays_in_range', 'actual_start', 'actual_end', 'actual_days', 'nan_count']

# Create function to get raw json return from FRED database
def get_series_json(series_id, start, end, api_key=API_KEY, file_type='json', url_base=URL_BASE):
    url = f'{url_base}?series_id={series_id}&observation_start={start}&observation_end={end}'
    url += f'&api_key={api_key}&file_type={file_type}'
    try:
        resp = requests.get(url)
        resp.raise_for_status()  # Raise exception if invalid response
        return resp
    except Exception as e:
        errmsg = resp.json()['error_message'].replace('series', f'series {series_id}')
        print(f'Error: {resp.status_code}\n{errmsg}')
        return None


# Create function to transform valid json response from FRED into a dataframe
def transform_series_json(resp, series_id):
    resp = resp.json()
    obs = pd.DataFrame(resp.pop('observations'))[['date', 'value']]
    obs['date'] = pd.to_datetime(obs['date'])
    obs.set_index('date', inplace=True)
    meta = pd.DataFrame({
        series_id: {'observation_start': resp['observation_start'],
            'observation_end': resp['observation_end'],
            'busdays_in_range': BUSDAYS_IN_RANGE,
            'actual_days': resp['count'],
            'actual_start': obs.index.min().date(),
            'actual_end': obs.index.max().date(),
            'nan_count': obs[obs.value == '.'].count().value}})
    meta = meta.reindex(META_INDEX)
    obs.loc[obs.value == '.'] = np.nan
    obs.columns = [series_id]
    obs[series_id] = obs[series_id].astype(float, errors='raise')
    obs.index = obs.index.date
    return obs, meta


# Create function to fill missing values in FRED series datafame
def fill_series_na(df):
    df.fillna(method='ffill', inplace=True)  # Fill missing values with last observation
    df.fillna(method='bfill', inplace=True)  # Then, fill with next observation
    return df


# Create a function to get a time series from FRED and return a clean dataframe
def get_series(series_id, start, end, api_key=API_KEY, file_type='json', fill_na=None):
    fill_na = True if fill_na is None else fill_na  # Default
    try:
        resp = get_series_json(series_id=series_id, start=start, end=end, api_key=api_key, file_type=file_type)
        df, meta = transform_series_json(resp, series_id=series_id)
        df = fill_series_na(df) if fill_na else df
    except Exception as e:
        print(f'Error retrieving {series_id}.\n{e}')
        return None
    return df, meta

# Convenience function to get multiple series at once
def get_multiple_series(series_list, start, end, fill_na=None):
    fill_na = True if fill_na is None else fill_na  # Default
    df_list = []
    meta_list = []
    for series in series_list:
        df, meta = get_series(series_id=series, start=start, end=end)
        df_list.append(df)
        meta_list.append(meta)
    print(f'\nDownloaded {len(df_list)} / {len(series_list)} series') 
    print(f'\nMeta Info on downloaded series: \n{pd.concat(meta_list, axis=1).to_markdown()}')
    print(f'\nCombined series dataframe: \n{pd.concat(df_list, axis=1).head().to_markdown()}')
    return pd.concat(df_list, axis=1), pd.concat(meta_list, axis=1)

In [56]:
series_list = ['DEXUSUK', 'DEXCAUS', 'DEXCHUS', 'DEXJPUS', 'DEXINUS', 'DEXSFUS']

dfs, metas = get_multiple_series(series_list=series_list, start=START, end=END, fill_na=True)



Downloaded 6 / 6 series

Meta Info on downloaded series: 
|                   | DEXUSUK    | DEXCAUS    | DEXCHUS    | DEXJPUS    | DEXINUS    | DEXSFUS    |
|:------------------|:-----------|:-----------|:-----------|:-----------|:-----------|:-----------|
| observation_start | 1983-01-01 | 1983-01-01 | 1983-01-01 | 1983-01-01 | 1983-01-01 | 1983-01-01 |
| observation_end   | 2022-12-31 | 2022-12-31 | 2022-12-31 | 2022-12-31 | 2022-12-31 | 2022-12-31 |
| busdays_in_range  | 10435      | 10435      | 10435      | 10435      | 10435      | 10435      |
| actual_start      | 1983-01-03 | 1983-01-03 | 1983-01-03 | 1983-01-03 | 1983-01-03 | 1983-01-03 |
| actual_end        | 2022-12-30 | 2022-12-30 | 2022-12-30 | 2022-12-30 | 2022-12-30 | 2022-12-30 |
| actual_days       | 10435      | 10435      | 10435      | 10435      | 10435      | 10435      |
| nan_count         | 395        | 395        | 456        | 395        | 403        | 404        |

Combined series dataframe: 
|           

In [60]:
dfs.describe()

,DEXUSUK,DEXCAUS,DEXCHUS,DEXJPUS,DEXINUS,DEXSFUS
count,"10,435.00","10,435.00","10,435.00","10,435.00","10,435.00","10,435.00"
mean,1.57,1.27,6.44,124.00,42.28,7.24
std,0.20,0.15,1.82,37.19,19.47,4.48
min,1.05,0.92,1.89,75.72,9.62,1.06
25%,1.44,1.17,5.72,105.60,29.00,3.06
50%,1.57,1.29,6.74,114.03,44.15,6.76
75%,1.67,1.37,8.28,128.64,54.92,10.21
max,2.11,1.61,8.74,262.80,82.95,19.04
